In [21]:
import sys

import dlib
from skimage import io

image_path = 'C:\\Users\\rafae\\Google Drive\\Mestrado_Data\\'

detector = dlib.get_frontal_face_detector()
win = dlib.image_window()

f = image_path + "p002\\02_19_ir2_203.bmp"

print("Processing file: {}".format(f))
img = io.imread(f)
# The 1 in the second argument indicates that we should upsample the image
# 1 time.  This will make everything bigger and allow us to detect more
# faces.
dets = detector(img, 1)
print("Number of faces detected: {}".format(len(dets)))
for i, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        i, d.left(), d.top(), d.right(), d.bottom()))

win.clear_overlay()
win.set_image(img)
win.add_overlay(dets)
dlib.hit_enter_to_continue()


# Finally, if you really want to you can ask the detector to tell you the score
# for each detection.  The score is bigger for more confident detections.
# The third argument to run is an optional adjustment to the detection threshold,
# where a negative value will return more detections and a positive value fewer.
# Also, the idx tells you which of the face sub-detectors matched.  This can be
# used to broadly identify faces in different orientations.
if (len(f) > 0):
    img = io.imread(f)
    dets, scores, idx = detector.run(img, 1, -1)
    for i, d in enumerate(dets):
        print("Detection {}, score: {}, face_type:{}".format(
            d, scores[i], idx[i]))

Processing file: C:\Users\rafae\Google Drive\Mestrado_Data\p002\02_19_ir2_203.bmp
Number of faces detected: 0
Detection [(86, 96) (175, 186)], score: -0.13082002054973207, face_type:1.0


In [ ]:
import sys
import os
import dlib
import glob
from skimage import io

if len(sys.argv) != 4:
    print(
        "Call this program like this:\n"
        "   ./face_recognition.py shape_predictor_5_face_landmarks.dat dlib_face_recognition_resnet_model_v1.dat ../examples/faces\n"
        "You can download a trained facial shape predictor and recognition model from:\n"
        "    http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2\n"
        "    http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2")
    exit()

image_path = 'C:\\Users\\rafae\\Google Drive\\Mestrado_Data\\'
f = image_path + "p002\\02_19_ir2_203.bmp"

predictor_path = sys.argv[1]
face_rec_model_path = 'C:\\Users\\rafae\\Downloads\\dlib_face_recognition_resnet_model_v1.dat'
faces_folder_path = image_path

# Load all the models we need: a detector to find the faces, a shape predictor
# to find face landmarks so we can precisely localize the face, and finally the
# face recognition model.
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

win = dlib.image_window()

# Now process all the images
#for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
print("Processing file: {}".format(f))
img = io.imread(f)

win.clear_overlay()
win.set_image(img)

# Ask the detector to find the bounding boxes of each face. The 1 in the
# second argument indicates that we should upsample the image 1 time. This
# will make everything bigger and allow us to detect more faces.
dets = detector(img, 1)
print("Number of faces detected: {}".format(len(dets)))

# Now process each face we found.
for k, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    # Get the landmarks/parts for the face in box d.
    shape = sp(img, d)
    # Draw the face landmarks on the screen so we can see what face is currently being processed.
    win.clear_overlay()
    win.add_overlay(d)
    win.add_overlay(shape)

    # Compute the 128D vector that describes the face in img identified by
    # shape.  In general, if two face descriptor vectors have a Euclidean
    # distance between them less than 0.6 then they are from the same
    # person, otherwise they are from different people. Here we just print
    # the vector to the screen.
    face_descriptor = facerec.compute_face_descriptor(img, shape)
    print(face_descriptor)
    # It should also be noted that you can also call this function like this:
    #  face_descriptor = facerec.compute_face_descriptor(img, shape, 100)
    # The version of the call without the 100 gets 99.13% accuracy on LFW
    # while the version with 100 gets 99.38%.  However, the 100 makes the
    # call 100x slower to execute, so choose whatever version you like.  To
    # explain a little, the 3rd argument tells the code how many times to
    # jitter/resample the image.  When you set it to 100 it executes the
    # face descriptor extraction 100 times on slightly modified versions of
    # the face and returns the average result.  You could also pick a more
    # middle value, such as 10, which is only 10x slower but still gets an
    # LFW accuracy of 99.3%.


    dlib.hit_enter_to_continue()